In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import matplotlib 

In [ ]:
def domino(lol):
    # Takes a list (length n) of lists (length 2)
    # and returns a list of indices order,
    # such that lol[order[i]] and lol[order[i+1]]
    # have at least one element in common.
    # If that is not possible, multiple
    # domino chains will be created.
    # This works in a greedy way.
    
    n = len(lol)
    order = [0] # Greedy 
    link = lol[0][-1]
    links = [lol[0][0],lol[0][1]]
    while len(order)<n:
        for i in [j for j in range(n) if not j in order]:
            if link in lol[i]: # They connect
                order.append(i) # Save the id of the "stone"
                link = lol[i][0] if not(lol[i][0]==link) else lol[i][1] # The new link is the other element 
                links.append(link)
                break
    return order,links[:-1]

In [ ]:
prefix="s"
xyz = np.loadtxt(f"{prefix}points.txt")
w = np.loadtxt(f"{prefix}weights.txt")
neighbours = np.loadtxt(f"{prefix}neighbours.txt",dtype=int)-1 # julia starts at 1
triangles = np.loadtxt(f"{prefix}triangles.txt",dtype=int)-1 # julia starts at 1 
nq = len(w)
steps = 1 # for colorbar

w = np.round(w,4)
print(w)

In [ ]:
plt.style.use("kitishblank")
fig = plt.figure(figsize=plt.figaspect(0.8)) #Adjusts the aspect ratio and enlarges the figure (text does not enlarge)
ax = fig.gca(projection='3d')
cmap = matplotlib.cm.get_cmap('plasma')
ww = np.sort(np.unique(w))
bounds = ww
bounds = np.insert(bounds,0,-1)
bounds = np.insert(bounds,-1,1)
bounds = (bounds[1:]+bounds[:-1]) / 2


norm = plt.Normalize(np.min(w),np.max(w))

for center in range(nq):
    lol = [] # list of lists
    for i in neighbours[center,:]:
        if i>-1:
            lol.append(list(np.sort(triangles[i,triangles[i,:] != center])))
    order,links = domino(lol)
 
    neighx = [xyz[j,0] for j in links]
    neighy = [xyz[j,1] for j in links]
    neighz = [xyz[j,2] for j in links]

    # Get the actual hexagon that surrounds a center point
    x = []
    y = []
    z = []
    for i in range(len(order)):
        x.append((xyz[center,0]+neighx[i]) / 2)
        x.append((xyz[center,0]+neighx[i]+neighx[(i+1)%len(order)])/3)

        y.append((xyz[center,1]+neighy[i]) / 2)
        y.append((xyz[center,1]+neighy[i]+neighy[(i+1)%len(order)])/3)
        
        z.append((xyz[center,2]+neighz[i]) / 2)
        z.append((xyz[center,2]+neighz[i]+neighz[(i+1)%len(order)])/3)
        
    # Project onto sphere
    #for i in range(len(order)):
    #    tmp = (x[i]**2+y[i]**2+z[i]**2)**0.5
    #    x[i]/=tmp
    #    y[i]/=tmp
    #    z[i]/=tmp
    
    verts = [list(zip(x,y,z))]
    
    rgb = cmap(norm(w[center]))
    
    ax.add_collection3d(Poly3DCollection(verts,facecolor = rgb,linewidth=0.2,edgecolor="k"))
    p= ax.scatter(0*xyz[center,0],0*xyz[center,1],0*xyz[center,2],color=rgb,cmap=cmap)
l = 0.7
ax.set_xlim([-l,l])
ax.set_ylim([-l,l])
ax.set_zlim([-l,l])
plt.tight_layout()
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
plt.axis("off")
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

centers = (bounds[1:] + bounds[:-1])/2
cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),boundaries = bounds,ticks=centers,ax=ax,shrink = 0.6)
cbar.ax.set_yticklabels(["{:06.5f}".format(bi) for bi in bounds])
cbar.ax.set_title("Quadrature \n weight")
#ax.view_init(elev = 45, azim =60)#45/2, 45/2)

fig.tight_layout()
fig.savefig("{}{}.pdf".format(prefix,nq))

In [ ]:
ww